In [81]:
import mediapipe as mp

import pandas as pd 

import numpy as np

import cv2

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

import pyautogui

import time


In [2]:
hand_landmarks = [11,13,15,17,19,21,12,14,16,18,20,22]

classes = {0 : "None" , 1 :"up" , 2 : "down", 3 : "left", 4 : "right"  }

mp_hos = mp.solutions.holistic

mp_draw= mp.solutions.drawing_utils


TRAIN A SKLEARN MODEL USING COLLECTED DATA

In [113]:
data = pd.read_csv("posture_class.csv")

x = np.array(data.drop(["class"],axis='columns'))

y = np.array(data['class'])

In [117]:
train_x , test_x , train_y ,test_y = train_test_split(x , y,test_size= 0.2 )

In [118]:
model1 = LogisticRegression(max_iter = 10000)
model1.fit(train_x,train_y)

LogisticRegression(max_iter=10000)

In [119]:
model1.score(test_x,test_y)

0.9727410398788491

DETECTION FROM WEBCAM

In [120]:
rescent_values = np.array([ 0 for i in range(5)])
rescent_values

array([0, 0, 0, 0, 0])

In [ ]:

cap = cv2.VideoCapture(0)


output = cv2.VideoWriter( 
        "output.mp4", cv2.VideoWriter_fourcc(*'mp4v'), cap.get(cv2.CAP_PROP_FPS), (480, 640)) 

display_text = "None"


n = len(hand_landmarks)*4
with mp_hos.Holistic(min_detection_confidence = 0.5 , min_tracking_confidence= 0.5) as detect:

    while cap.isOpened():
        _ , frame = cap.read()
        
        try:
            
            detection_results = detect.process(frame)
            landmarks = detection_results.pose_landmarks.landmark
    
            
            mp_draw.draw_landmarks(frame , detection_results.pose_landmarks , mp_hos.HAND_CONNECTIONS)
    
            s = np.array([[landmarks[i].x , landmarks[i].y , landmarks[i].z , landmarks[i].visibility] for i in hand_landmarks]).flatten().reshape(-1,n)
    
            p = model1.predict(s)[0]

            rescent_values = rescent_values[1:]
            rescent_values = np.append(rescent_values , int(p))
            if(np.max(rescent_values) == np.min(rescent_values)):
                display_text = classes[rescent_values[0]]
                if(classes[rescent_values[0]] != "None"):
                    control_game(rescent_values[0])
                    rescent_values = np.array([ 0 for i in range(5)])
                    
           
                    
                
    
            cv2.putText(frame,display_text,(30,30) ,cv2.FONT_HERSHEY_SIMPLEX , 1,(0,0,255) ,3)
                

        except:
            pass
       
        
        
    

        output.write(frame)
        cv2.imshow("detections" , frame)
    
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()   
output.release()
cv2.destroyAllWindows()

CONTROL THE GAME USING MOVEMENTS OF THE MOUSE (adjust position as per your game mouse configurations)

In [125]:


def control_game(move):

    default_pos = (538,331)
    pyautogui.moveTo(default_pos)
    if(move == 1):
        #moving up
        pyautogui.dragTo(538,231)
        return 0

    if(move == 2):
        #moving down
        pyautogui.dragTo(538,431)
        return 0
        
    if(move == 3):
        #moving left
        pyautogui.dragTo(438,331)
        return 0

    if(move == 4):
        #moving right
        pyautogui.dragTo(638,331)
        return 0

        

In [52]:
pyautogui.position()

Point(x=669, y=335)

In [51]:
pyautogui.moveTo(538,331)

In [95]:
time.sleep(7)

pyautogui.dragTo(538,431)